In [39]:
from skyfield.api import load, EarthSatellite, wgs84
from spacetrack import SpaceTrackClient
from io import StringIO  # Import StringIO
import time
import matplotlib.pyplot as plt
import tqdm
import pandas as pd
import time

# Example

In [40]:

# 1. Setup timescale and load TLE
ts = load.timescale()

# Example TLE (International Space Station - ISS)
line1 = '1 25544U 98067A   24021.57946450  .00016462  00000-0  29550-3 0  9990'
line2 = '2 25544  51.6416 111.9329 0004546 128.5284 316.4800 15.49817757435427'

satellite = EarthSatellite(line1, line2, 'ISS', ts)

# 2. Specify the time you want the position for
t = ts.now()

# 3. Propagate the satellite to that time
geocentric = satellite.at(t)

# 4. GET COORDINATES
# A. Geocentric Cartesian (X, Y, Z in km from Earth's center)
x, y, z = geocentric.position.km
print(f"Cartesian (km): X={x:.2f}, Y={y:.2f}, Z={z:.2f}")

# B. Geographic (Lat, Lon, Elevation)
# We use the WGS84 ellipsoid model for Earth
subpoint = wgs84.subpoint(geocentric)

print(f"Latitude:  {subpoint.latitude.degrees:.4f}°")
print(f"Longitude: {subpoint.longitude.degrees:.4f}°")
print(f"Altitude:  {subpoint.elevation.km:.2f} km")

Cartesian (km): X=4829.72, Y=4056.69, Z=2132.54
Latitude:  18.9060°
Longitude: -171.6150°
Altitude:  282.22 km


In [41]:
def tle_to_dataframe(tle_data_string):
    """
    Parses a string containing TLE data into a Pandas DataFrame.

    Args:
        tle_data_string (str): A string containing TLE data in the standard two-line format.

    Returns:
        pandas.DataFrame: A DataFrame containing the parsed TLE data.  Returns an empty
                        DataFrame if the input string is empty or only contains headers.
    """
    # Ensure the input is a string
    if not isinstance(tle_data_string, str):
        raise TypeError("Input must be a string.")

    # Use StringIO to treat the string as a file
    tle_file = StringIO(tle_data_string)

    # Read the lines from the string
    lines = tle_file.readlines()

    if not lines:
        return pd.DataFrame()  # Return an empty DataFrame for empty input

    # Initialize lists to store the data
    data = []
    header_names = [
        "Satellite Name", "Epoch", 
        "First Derivative Mean Motion", "Second Derivative Mean Motion",
        "BSTAR Drag Term", "Element Set Number", "Inclination (degrees)",
        "Right Ascension of the Ascending Node (degrees)", "Eccentricity",
        "Argument of Perigee (degrees)", "Mean Anomaly (degrees)",
        "Mean Motion (revolutions per day)", "Revolution Number at Epoch"
    ]

    # Process TLE lines in pairs
    for i in range(0, len(lines) - 1, 2):
        try:
            line1 = lines[i].strip()
            line2 = lines[i + 1].strip()

            # Extract data from line 1
            satellite_name = line1[2:18].strip()
            epoch = line1[18:32].strip()
            first_derivative_mean_motion = float(line1[33:43].strip())

            # Handle potential errors in these fields
            second_derivative_mean_motion_str = line1[44:52].replace('-', '-0').strip()
            if second_derivative_mean_motion_str:
                try:
                    second_derivative_mean_motion = float(second_derivative_mean_motion_str)
                except ValueError:
                    second_derivative_mean_motion = 0.0
            else:
                second_derivative_mean_motion = 0.0

            bstar_drag_term_str = line1[53:61].replace('-', '-0').strip()
            if bstar_drag_term_str:
                try:
                    bstar_drag_term = float(bstar_drag_term_str)
                except ValueError:
                    bstar_drag_term = 0.0
            else:
                bstar_drag_term = 0.0
            element_set_number = int(line1[64:68].strip())

            # Extract data from line 2
            inclination = float(line2[8:16].strip())
            raan = float(line2[17:25].strip())
            eccentricity = float("0." + line2[26:33].strip())  # Add leading zero
            arg_of_perigee = float(line2[34:42].strip())
            mean_anomaly = float(line2[43:51].strip())
            mean_motion = float(line2[52:63].strip())
            revolution_number = int(line2[63:68].strip())

            # Append the data as a list
            data.append([
                satellite_name,  epoch,
                first_derivative_mean_motion, second_derivative_mean_motion,
                bstar_drag_term, element_set_number, inclination, raan, eccentricity,
                arg_of_perigee, mean_anomaly, mean_motion, revolution_number
            ])
        except (ValueError, IndexError) as e:
            print(f"Error parsing TLE lines {i+1}-{i+2}: {e}. Skipping these lines.")
            # Handle errors in parsing (e.g., malformed TLE data)
            continue

    # Create the Pandas DataFrame
    df = pd.DataFrame(data, columns=header_names)
    return df

# df = tle_to_dataframe(tle_data)
# # print(tle_data)
# df.head()
# Using Pred and Test data

In [42]:
def tle_to_lines(tle_data_string):
    """
    Parses a string containing TLE data into a Pandas DataFrame.

    Args:
        tle_data_string (str): A string containing TLE data in the standard two-line format.

    Returns:
        pandas.DataFrame: A DataFrame containing the parsed TLE data.  Returns an empty
                        DataFrame if the input string is empty or only contains headers.
    """
    # Ensure the input is a string
    if not isinstance(tle_data_string, str):
        raise TypeError("Input must be a string.")

    # Use StringIO to treat the string as a file
    tle_file = StringIO(tle_data_string)

    # Read the lines from the string
    lines = tle_file.readlines()

    if not lines:
        return pd.DataFrame()  # Return an empty DataFrame for empty input

    # Initialize lists to store the data
    data = []
    header_names = [
        "Satellite Name", "Epoch", 
        "First Derivative Mean Motion", "Second Derivative Mean Motion",
        "BSTAR Drag Term", "Element Set Number", "Inclination (degrees)",
        "Right Ascension of the Ascending Node (degrees)", "Eccentricity",
        "Argument of Perigee (degrees)", "Mean Anomaly (degrees)",
        "Mean Motion (revolutions per day)", "Revolution Number at Epoch"
    ]

    # Process TLE lines in pairs
    line_data = []
    for i in range(0, len(lines) - 1, 2):
        try:
            line1 = lines[i].strip()
            line2 = lines[i + 1].strip()

            line_data.append([line1, line2])
        except (ValueError, IndexError) as e:
            print(f"Error parsing TLE lines {i+1}-{i+2}: {e}. Skipping these lines.")
            # Handle errors in parsing (e.g., malformed TLE data)
            continue
            
    return line_data

    # Create the Pandas DataFrame
#     df = pd.DataFrame(data, columns=header_names)
#     return df

# df = tle_to_dataframe(tle_data)
# # print(tle_data)
# df.head()


In [43]:
with open("result_TLE/satellites_pred.tle", "r") as f:
    pred_data = f.read()

print("Clean TLE file saved as 'satellites.tle'")

Clean TLE file saved as 'satellites.tle'


In [44]:
with open("result_TLE/satellites_test.tle", "r") as f:
    test_data = f.read()

print("Clean TLE file saved as 'satellites.tle'")

Clean TLE file saved as 'satellites.tle'


In [45]:
len(tle_to_lines(data))

372

In [46]:
tle_to_lines(pred_data)[0]

['1 44714U 19074B   24229.32060812 .00010575  00000-0  00000-0 0  9990',
 '2 44714  53.0528 225.3163 0001409  87.3589 272.6483 15.06408427262909']

# Get position

In [52]:
pred_row_data = []
limit_req = 10
sat_name = 'STARLINK-1008'
for [line1, line2] in tqdm.tqdm(tle_to_lines(pred_data)[:limit_req]):
    

    satellite = EarthSatellite(line1, line2, sat_name, ts)

    # 2. Specify the time you want the position for
    t = ts.now()

    # 3. Propagate the satellite to that time
    geocentric = satellite.at(t)

    # 4. GET COORDINATES
    # A. Geocentric Cartesian (X, Y, Z in km from Earth's center)
    x, y, z = geocentric.position.km
#     print(f"Cartesian (km): X={x:.2f}, Y={y:.2f}, Z={z:.2f}")

    # B. Geographic (Lat, Lon, Elevation)
    # We use the WGS84 ellipsoid model for Earth
    subpoint = wgs84.subpoint(geocentric)

#     print(f"Latitude:  {subpoint.latitude.degrees:.4f}°")
#     print(f"Longitude: {subpoint.longitude.degrees:.4f}°")
#     print(f"Altitude:  {subpoint.elevation.km:.2f} km")
    
    pred_row_data.append([x,y,z,subpoint.latitude.degrees,subpoint.longitude.degrees,subpoint.elevation.km])
    
    time.sleep(3)
#     break

100%|██████████| 10/10 [00:30<00:00,  3.01s/it]


In [56]:
pred_df = pd.DataFrame(pred_row_data, columns=['X','Y','Z','Lat','Long','Ele'])
pred_df

,X,Y,Z,Lat,Long,Ele
0,-5757.468568,-756.877200,3765.622461,32.978622,-36.306931,549.234811
1,-5472.585296,-418.956844,4215.150922,37.549140,-39.426877,550.186398
2,-5449.429185,-141.968189,4263.229283,38.053547,-42.319313,550.321089
3,-5479.558758,-646.130036,4177.258445,37.154323,-37.109121,550.067993
4,-6039.004515,-2180.380330,2587.908751,21.937392,-24.002356,547.320297
5,-5862.566982,-1059.972068,3522.914263,30.608263,-33.613470,548.657668
6,-5939.456719,-1974.749477,2955.854078,25.276418,-25.490302,547.720239
7,-3701.121381,2142.660479,5438.809512,51.868424,-73.845829,553.865452
8,-4383.799413,1442.610782,5155.297840,48.202913,-62.048046,552.949433
9,-4918.201106,589.301106,4831.689361,44.319944,-50.712051,551.883819


In [54]:
test_row_data = []
limit_req = 10

sat_name = 'STARLINK-1008'
for [line1, line2] in tqdm.tqdm(tle_to_lines(test_data)[:limit_req]):
    

    satellite = EarthSatellite(line1, line2, sat_name, ts)

    # 2. Specify the time you want the position for
    t = ts.now()

    # 3. Propagate the satellite to that time
    geocentric = satellite.at(t)

    # 4. GET COORDINATES
    # A. Geocentric Cartesian (X, Y, Z in km from Earth's center)
    x, y, z = geocentric.position.km
#     print(f"Cartesian (km): X={x:.2f}, Y={y:.2f}, Z={z:.2f}")

    # B. Geographic (Lat, Lon, Elevation)
    # We use the WGS84 ellipsoid model for Earth
    subpoint = wgs84.subpoint(geocentric)

#     print(f"Latitude:  {subpoint.latitude.degrees:.4f}°")
#     print(f"Longitude: {subpoint.longitude.degrees:.4f}°")
#     print(f"Altitude:  {subpoint.elevation.km:.2f} km")
    
    test_row_data.append([x,y,z,subpoint.latitude.degrees,subpoint.longitude.degrees,subpoint.elevation.km])
    
    time.sleep(2)
#     break

100%|██████████| 10/10 [00:20<00:00,  2.01s/it]


In [57]:
test_df = pd.DataFrame(pred_row_data, columns=['X','Y','Z','Lat','Long','Ele'])
test_df

,X,Y,Z,Lat,Long,Ele
0,-5757.468568,-756.877200,3765.622461,32.978622,-36.306931,549.234811
1,-5472.585296,-418.956844,4215.150922,37.549140,-39.426877,550.186398
2,-5449.429185,-141.968189,4263.229283,38.053547,-42.319313,550.321089
3,-5479.558758,-646.130036,4177.258445,37.154323,-37.109121,550.067993
4,-6039.004515,-2180.380330,2587.908751,21.937392,-24.002356,547.320297
5,-5862.566982,-1059.972068,3522.914263,30.608263,-33.613470,548.657668
6,-5939.456719,-1974.749477,2955.854078,25.276418,-25.490302,547.720239
7,-3701.121381,2142.660479,5438.809512,51.868424,-73.845829,553.865452
8,-4383.799413,1442.610782,5155.297840,48.202913,-62.048046,552.949433
9,-4918.201106,589.301106,4831.689361,44.319944,-50.712051,551.883819


In [58]:
df_abs_diff = (pred_df - test_df).abs()

# 2. Compute the Average for each column
column_avg_diff = df_abs_diff.mean()

# 3. Compute the Global Average (a single number for the whole DF)
global_avg_diff = df_abs_diff.values.mean()

print("Average Difference per Column:")
print(column_avg_diff)

Average Difference per Column:
X       0.0
Y       0.0
Z       0.0
Lat     0.0
Long    0.0
Ele     0.0
dtype: float64


In [59]:
df_abs_diff

,X,Y,Z,Lat,Long,Ele
0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0
